<font size=7>Reliability of the salience features and object categories<font> 

<font size=3>Finally, we want to look at the split-half reliability of the salience features and the object categories in our data set. Each file contains two columns for the values from even and odd images, each row contains the data of one subject. Calculate the split-half reliability for all saliency features and object categories. How do you assess the reliabilities?<font>


In [6]:
import os, glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy.io
import scipy.stats
from scipy.ndimage import gaussian_filter
#from ipywidgets import interact, Button
import ipywidgets as widgets
from IPython.display import display, HTML
from PIL import Image
#import solara
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')

RESULTS_FOLDER_FIX = 'data_task_C/data_fix/'
RESULTS_FOLDER_SACC = 'data_task_C/data_sacc/'
SALIENCY_MAP_DICT = 'data_task_F/saliency_map_dict.npy'
OSIE_FILE = 'data_task_F/attrs_OSIE_40.mat'

In [8]:
def extract_semantic_saliency_fix(subject_num):
    # Add necessary path
    fix_data_path = (
        #"/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/data/derivatives/"
        RESULTS_FOLDER_FIX
        + subject_num
        + "_fix_data.csv"
    )

    OSIE_data_path = OSIE_FILE#"/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/OSIE_40/attrs_OSIE_40.mat"

    saliency_map_path = SALIENCY_MAP_DICT#"/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/OSIE_40/saliency_map_dict.npy"

    # Load the MATLAB .mat file containing the data
    fix_data = pd.read_csv(fix_data_path, sep=",")
    # mat_data = loadmat(OSIE_data_path)
    mat_dict = loadmat(OSIE_data_path, simplify_cells=True)
    attrs_OSIE_40 = mat_dict["attrs_OSIE_40"]
    saliency_map_dict = np.load(
        saliency_map_path,
        allow_pickle="TRUE",
    ).item()

    # print(saliency_map_dict)
    # df = pd.read_table("/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/pilot_res/pp_scan_2024_03_21_16_08.asc")
    # ascii_grid = np.loadtxt("/Users/patrikpolgari/Documents/Postdoc_Marburg/student_supervision/Teaching/B-NP2_Übung/Code/Task_C/pilot_res/pp_scan_2024_03_21_16_08.asc", skiprows=100)

    # Constants and parameters
    dims = [0, 1, 5, 7, 10]
    # Header = ["Text", "Face", "Taste", "Motion", "Touched"]

    im_width, im_height = 800, 600

    im_width_real, im_height_real = 1146, 876  # real size
    exp_scale_width = im_width_real / im_width  # 1.4325
    exp_scale_height = im_height_real / im_height  # 1.46
    screen_width, screen_height = 1920, 1080
    # screen_x_cm, screen_y_cm, vp_dist_cm = 51.5, 29.0, 60
    # sample_freq = 1000
    # XPIX2DEG = np.arctan((screen_x_cm / screen_width) / vp_dist_cm) * (180 / np.pi)
    # YPIX2DEG = np.arctan((screen_y_cm / screen_height) / vp_dist_cm) * (180 / np.pi)
    # folder_name = f"e1v{subject_num}b1"
    fixations = {}

    # im_num = '1001.jpg'
    # block_num = 1

    BLOCK_NUMS = 40
    for block_num in range(0, BLOCK_NUMS):
        # block_num = 1
        image_name = attrs_OSIE_40[block_num]["img"]
        # objs = attrs_OSIE_40[block_num]["objs"]

        fix_data_block = fix_data[fix_data["img"] == image_name]
        Cfix = len(fix_data_block)  # fixation count
        Dfix = list(fix_data_block["dur"])  # fixation durations
        Xfix = list(
            ((fix_data_block["axp"] - (screen_width / 2)) / exp_scale_width) + (im_width / 2)
        )
        # Xfix = Xfix - (screen_width/2) #/ exp_scale_width)
        # Xfix = Xfix + (im_width/2)
        # Xfix_norm = Xfix/exp_scale_width # X coord
        Yfix = list(
            ((fix_data_block["ayp"] - (screen_height / 2)) / exp_scale_height) + (im_height / 2)
        )  # / exp_scale_height)  # Y coord
        Order = list(fix_data_block["fix_order"])
        # for im_num in pd.unique(fix_data['img']):

        # Analyze semantic content
        sem = np.zeros((Cfix, len(dims)))

        for d in range(len(dims)):
            map_sum = np.zeros((im_height, im_width))
            # map_ori = []
            for obj in range(len(attrs_OSIE_40[block_num]["objs"])):
                # obj = 0
                if attrs_OSIE_40[block_num]["objs"][obj]["features"][dims[d]] > 0:
                    # print(attrs_OSIE_40[block_num-1]['objs'][obj]['features'][dims[d]])
                    map_ori = attrs_OSIE_40[block_num]["objs"][obj]["map"]
                    # print(map_ori)
                    map_sum += map_ori
                    # print(map_sum)
            for f in range(Cfix):
                sem[f, d] = 0
                if 0 < round(Xfix[f]) <= im_width and 0 < round(Yfix[f]) <= im_height:
                    sem[f, d] = map_sum[int(round(Yfix[f])), int(round(Xfix[f]))] * Dfix[f]

        sem_df = pd.DataFrame(sem)

        # Analyze saliency content
        sal = np.zeros((Cfix, len(saliency_map_dict[image_name])))
        list_of_sal_maps = list(saliency_map_dict[image_name].keys())

        for i, sal_map in enumerate(list_of_sal_maps):
            # print(i, sal_map)
            sal_map_ori = saliency_map_dict[image_name][sal_map]

            for f in range(Cfix):
                sal[f, i] = 0
                if 0 < round(Xfix[f]) <= im_width and 0 < round(Yfix[f]) <= im_height:
                    # sal[f, i] = sal_map_ori[int(round(Yfix[f])), int(round(Xfix[f]))] * Dfix[f]
                    sal[f, i] = (
                        np.sum(sal_map_ori < sal_map_ori[int(round(Yfix[f])), int(round(Xfix[f]))])
                        / sal_map_ori.size
                    )  # * Dfix[f]

        sal_df = pd.DataFrame(sal, columns=list_of_sal_maps)

        fixations[image_name] = {
            "X": Xfix,
            "Y": Yfix,
            # "Onset": Tfix,
            "Duration": Dfix,
            # "Amplitude": Asac,
            "Sem": sem_df,
            "Sal": sal_df,
            "Order": Order,
        }

    return fixations



def extract_fix_split_half():
    # fixations_odd = {}
    # fixations_even = {}
    fix_dataframe_group = pd.DataFrame()
    subject_list = ["zt", "pp", "jp"]
    for subject_num in subject_list:
        # subject_num='zt'
        # fixations = extract_semantic_fix(subject_num)
        fixations = extract_semantic_saliency_fix(subject_num)
        # Constants and parameters
        dims = [0, 1, 5, 7, 10]
        header_sem1 = ["Text1", "Face1", "Taste1", "Motion1", "Touched1"]
        header_sal1 = ["Saliency1", "Intensity1", "Color1", "Orientation1"]
        header_sem2 = ["Text2", "Face2", "Taste2", "Motion2", "Touched2"]
        header_sal2 = ["Saliency2", "Intensity2", "Color2", "Orientation2"]
        header_tot = header_sem1 + header_sem2 + header_sal1 + header_sal2

        first_fix = []
        all_fix_sem_1 = pd.DataFrame()
        all_fix_sal_1 = pd.DataFrame()
        all_fix_sem_2 = pd.DataFrame()
        all_fix_sal_2 = pd.DataFrame()
        # d_fix = []
        all_dur_1 = []  # pd.DataFrame()
        all_dur_2 = []
        fixation_keys = list(fixations.keys())

        for i, key in enumerate(fixation_keys):
            # key = '1001.jpg'
            if fixations[key] is not None:
                # sem
                df_sem = pd.DataFrame(fixations[key]["Sem"])
                df_sal = fixations[key]["Sal"]
                df_dur = fixations[key]["Duration"]

                if i % 2 == 0:
                    all_fix_sem_2 = pd.concat([all_fix_sem_2, df_sem])
                    all_dur_2 += df_dur
                    all_fix_sal_2 = pd.concat([all_fix_sal_2, df_sal])

                else:
                    all_fix_sem_1 = pd.concat([all_fix_sem_1, df_sem])
                    all_dur_1 += df_dur
                    all_fix_sal_1 = pd.concat([all_fix_sal_1, df_sal])

        # first_prop = np.sum(np.sign(first_fix), axis=0) / len(first_fix)
        # all_prop = np.sum(all_fix, axis=0) / np.sum(d_fix)
        all_prop_sem_1 = all_fix_sem_1.sum() / sum(all_dur_1)
        all_prop_sem_2 = all_fix_sem_2.sum() / sum(all_dur_2)
        all_prop_sal_1 = all_fix_sal_1.mean()
        all_prop_sal_2 = all_fix_sal_2.mean()

        # sem_fix_data = pd.DataFrame((np.array(header), first_prop, np.array(all_prop))).T.rename(columns={0:'header', 1:'first_prop', 2:'all_prop'})
        fix_data_ind = {
            # "header": header_sem1 + header_sem2 + header_sal1 + header_sal2,
            # "first_prop": list(first_prop),
            "all_prop": list(all_prop_sem_1)
            + list(all_prop_sem_2)
            + list(all_prop_sal_1)
            + list(all_prop_sal_2),
        }
        fix_dataframe_ind = pd.DataFrame.from_dict(
            fix_data_ind, orient="index", columns=header_tot
        )  # , columns=["header", "all_prop"]).T
        fix_dataframe_ind["sub_num"] = subject_num
        # fix_dataframe_ind(columns="header")
        fix_dataframe_group = pd.concat([fix_dataframe_group, fix_dataframe_ind])
        
    return fix_dataframe_group



In [9]:
df = extract_fix_split_half()
print(df)

             Text1     Face1    Taste1   Motion1  Touched1     Text2  \
all_prop  0.088915  0.411927  0.090332  0.303647  0.101175  0.156961   
all_prop  0.025727  0.139064  0.189396  0.203593  0.059777  0.074578   
all_prop  0.098437  0.218040  0.123872  0.218894  0.117594  0.171854   

             Face2    Taste2   Motion2  Touched2  Saliency1  Intensity1  \
all_prop  0.418974  0.041790  0.243774  0.136339   0.663116    0.660642   
all_prop  0.132261  0.041072  0.180172  0.065554   0.675862    0.661572   
all_prop  0.321564  0.043318  0.196337  0.101642   0.645369    0.632767   

            Color1  Orientation1  Saliency2  Intensity2    Color2  \
all_prop  0.560854      0.682619   0.677480    0.639840  0.572125   
all_prop  0.573157      0.693457   0.662665    0.621600  0.588453   
all_prop  0.555469      0.683669   0.660444    0.654477  0.543115   

          Orientation2 sub_num  
all_prop      0.695812      zt  
all_prop      0.654590      pp  
all_prop      0.682514      jp  


In [13]:
Color1 = df['Color1']
Color2 = df['Color2']
print(Color1)

R_col, p_col = scipy.stats.pearsonr(Color1, Color2)
print(R_col, p_col)


reliability = (2*R_col)/(1+R_col)
print(reliability)

all_prop    0.560854
all_prop    0.573157
all_prop    0.555469
Name: Color1, dtype: float64
0.9278293743636787 0.24334507513265086
0.9625637898270211


In [15]:
Ori1 = df['Orientation1']
Ori2 = df['Orientation2']
print(Ori1, Ori2)

R_col, p_col = scipy.stats.pearsonr(Ori1, Ori2)
print(R_col, p_col)


reliability = (2*R_col)/(1+R_col)
print(reliability)

all_prop    0.682619
all_prop    0.693457
all_prop    0.683669
Name: Orientation1, dtype: float64 all_prop    0.695812
all_prop    0.654590
all_prop    0.682514
Name: Orientation2, dtype: float64
-0.972836525216547 0.14872219778771217
-71.62828268268255
